# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [99]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [100]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [101]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [102]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [103]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [104]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [105]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [106]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

"""
Create table for event data processing

"""

query = "CREATE TABLE IF NOT EXISTS event_data_table "
query = query + """(artist varchar, song_title varchar, length float, sessionid int, iteminsession int, PRIMARY KEY(sessionid, iteminsession))"""
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [107]:
"""
Note: This block for checking variable.
What checking ?
1. checking in file columns by pandas csv reader
1.1. check column index before insert table or check index in list ('line' variable)
2. when checking, beware existing table >> must drop !
3. datatype in csv data table will match above block (CREATE TABLE BLOCK)

"""

# query = "drop table event_data_table "

# try:
#     session.execute(query)
# except Exception as e:
#     print(e)

# xxx = pd.read_csv(file)
# xxx.tail()


# file = 'event_datafile_new.csv'

# with open(file, encoding = 'utf8') as f:
#     csvreader = csv.reader(f)
#     next(csvreader) # skip header
#     for line in csvreader:
#          print(line)

"\nNote: This block for checking variable.\nWhat checking ?\n1. checking in file columns by pandas csv reader\n1.1. check column index before insert table or check index in list ('line' variable)\n2. when checking, beware existing table >> must drop !\n3. datatype in csv data table will match above block (CREATE TABLE BLOCK)\n\n"

In [108]:
"""
For Check values that corect !
"""

print(line)

test_data = pd.read_csv(file)
test_data.tail()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
6815,Sidewalk Prophets,Molly,F,0,Taylor,260.62322,free,"St. Louis, MO-IL",464,You Love Me Anyway (Album),35
6816,Rise Against,Molly,F,1,Taylor,221.17832,free,"St. Louis, MO-IL",464,Torches,35
6817,K'Naan,Molly,F,2,Taylor,220.49914,free,"St. Louis, MO-IL",464,Wavin' Flag,35
6818,Patrick Jumpen,Ryan,M,0,Smith,208.87465,free,"San Jose-Sunnyvale-Santa Clara, CA",480,Holiday,26
6819,Alicia Keys,Ryan,M,1,Smith,216.47628,free,"San Jose-Sunnyvale-Santa Clara, CA",480,Empire State Of Mind (Part II) Broken Down,26


In [109]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#

"""

Insert the data into created table with for loop and append for each row

"""
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "insert into event_data_table (artist, song_title, length, sessionid, iteminsession)"
        query = query + "values (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [111]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
##Add in the SELECT statement to verify the data was entered into the table

"""

Select with condition (sessionid = 338 and iteminsession = 4) for get artist name, song title, song length

"""

#Question 1
query = "SELECT * from event_data_table where sessionid = 338 and iteminsession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [112]:
"""
For Checking query (Select statement)
"""

test_data.loc[(test_data['itemInSession'] == 4) & (test_data['sessionId'] == 338)]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
444,Faithless,Ava,F,4,Robinson,495.3073,free,"New Haven-Milford, CT",338,Music Matters (Mark Knight Dub),50


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [113]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

"""
Create table for log data processing

"""

query = "CREATE TABLE IF NOT EXISTS log_data_table "
query = query + """(artist varchar, song_title varchar, firstname varchar, lastname varchar, 
userid int, sessionid int, iteminsession int, PRIMARY KEY((userid, sessionid), iteminsession))"""
try:
    session.execute(query)
except Exception as e:
    print(e)

In [114]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

"""

Insert the data into created table with for loop and append for each row

"""

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "insert into log_data_table (artist, song_title, firstname, lastname, userid, sessionid, iteminsession)"
        query = query + "values (%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3])))
                    

In [115]:
"""

Select with condition (userid = 10 and sessionid = 182 ORDER BY iteminsession ASC) for get artist name, song title, user's firstname and user's lastname

"""

#Question 2

query = "SELECT artist, song_title, firstname, lastname from log_data_table where userid = 10 and sessionid = 182 ORDER BY iteminsession ASC"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [116]:
"""
For Checking query (Select statement)
"""

test_data[(test_data['sessionId'] == 182) & (test_data['userId'] == 10)]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
4704,Down To The Bone,Sylvie,F,0,Cruz,333.76608,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Keep On Keepin' On,10
4705,Three Drives,Sylvie,F,1,Cruz,411.63710,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Greece 2000,10
4706,Sebastien Tellier,Sylvie,F,2,Cruz,377.73016,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Kilometer,10
4707,Lonnie Gordon,Sylvie,F,3,Cruz,181.21098,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Catch You Baby (Steve Pitron & Max Sanna Radio...,10


In [32]:
"""
For Check column index in the data
When checked, You can create and insert column correctly.
"""
# with open(file, encoding = 'utf8') as f:
#     csvreader = csv.reader(f)
#     next(csvreader) # skip header
#     for line in csvreader:
#         print(line)

# test_data = pd.read_csv(file)
# test_data.tail()

['Alicia Keys',
 'Ryan',
 'M',
 '1',
 'Smith',
 '216.47628',
 'free',
 'San Jose-Sunnyvale-Santa Clara, CA',
 '480',
 'Empire State Of Mind (Part II) Broken Down',
 '26']

In [117]:

"""
Create table for log data processing second time
Insert the data into created table with for loop and append for each row

"""

query = "CREATE TABLE IF NOT EXISTS log_data_table_2 "
query = query + """(artist varchar, song_title varchar, firstname varchar, lastname varchar, 
userid int, sessionid int, iteminsession int, PRIMARY KEY(song_title, firstname, lastname))"""
try:
    session.execute(query)
except Exception as e:
    print(e)


file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "insert into log_data_table_2 (artist, song_title, firstname, lastname, userid, sessionid, iteminsession)"
        query = query + "values (%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3])))

In [118]:
"""

Select with condition (song_title = 'All Hands Against His Own') for get user's firstname and user's lastname

"""

#Question 3

# artist, song_title, firstname, lastname, userid, sessionid, iteminsession

query = "SELECT firstname, lastname from log_data_table_2 where song_title = 'All Hands Against His Own' "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Sara Johnson
Tegan Levine


In [120]:
"""
For Checking query (Select statement)
"""

test_data[test_data['song'] == "All Hands Against His Own"]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
2792,The Black Keys,Tegan,F,25,Levine,196.91057,paid,"Portland-South Portland, ME",611,All Hands Against His Own,80
5135,The Black Keys,Sara,F,31,Johnson,196.91057,paid,"Winston-Salem, NC",152,All Hands Against His Own,95
6298,The Black Keys,Jacqueline,F,50,Lynch,196.91057,paid,"Atlanta-Sandy Springs-Roswell, GA",559,All Hands Against His Own,29


### Drop the tables before closing out the sessions

In [121]:
## TO-DO: Drop the table before closing out the sessions
query = "drop table event_data_table "

try:
    session.execute(query)
except Exception as e:
    print(e)


In [122]:
query = "drop table log_data_table "

try:
    session.execute(query)
except Exception as e:
    print(e)

In [123]:
query = "drop table log_data_table_2 "

try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [124]:
session.shutdown()
cluster.shutdown()